In [102]:
#| default_exp chat

In [103]:
# import getpass
# import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [104]:
# import getpass
# import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-3.5-turbo")

In [105]:
#| export 
from langchain_core.messages import HumanMessage, SystemMessage

from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [106]:
#| export 
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

helperPrompt  = """You are a job search assistant. 
You are helping a job searcher find jobs to apply for. From the list of jobs you have, you need to help the job searcher find the jobs that are relevant to them. You can ask the job searcher questions to help you understand what they are looking for.
"""

basePrompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            helperPrompt,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = basePrompt | model


In [107]:
requirementsPrompt  = """Current search requirements are: {
    "Job Title": "Software Engineer",
    "Location": "Remote",
    "Company": "Google"
}
"""

searchResultsPrompt = """
Current search results are: [
    {
        "Job Title": "Software Engineer",
        "Location": "Remote",
        "Company": "Google",
        "Status": "Pending"
    },
    {
        "Job Title": "Software Engineer",
        "Location": "Onsite",
        "Company": "Facebook",
        "Status": "Pending"
    },
    {
        "Job Title": "Software Engineer",
        "Location": "Remote",
        "Company": "Amazon",
        "Status": "Pending"
    }
]
"""

In [108]:
#| export
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory
)

from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")


In [109]:
#| export 
config = {"configurable": {"session_id": "abc4"}}

In [110]:
#| export
def remove_previous_RAG_info(session_id: str) -> dict:
    history = get_session_history(session_id)

    alteredHistory = []

    for message in history.messages:
        if type(message) is tuple:
            msgType, text = message
            if "Current search requirements are" not in text:
                if "Current search results are" not in text:
                    alteredHistory.append(message)    
        elif type(message) is HumanMessage:
            alteredHistory.append(message)
        else:
            alteredHistory.append(message)

    history.messages = alteredHistory

    return {"session_id": session_id, "messages": alteredHistory}

In [111]:
#| export
from langchain_core.prompts import ChatPromptTemplate

requirementsPrompt = ChatPromptTemplate.from_template(
"""Current search requirements are: {requirements}""")

def getRequirementsPrompt(requirements):
    return requirementsPrompt.format(requirements=requirements)


In [112]:
#| export
from langchain_core.prompts import ChatPromptTemplate

requirementsPrompt = ChatPromptTemplate.from_template(
"""Search results meeting those requirements are: {requirements}""")

def getSearchResultsPrompt(requirements):
    return requirementsPrompt.format(requirements=requirements)


In [113]:
#| export

def getChatResponse(input, requirements, searchResults):
    # Remove the previous RAG info, so we can re-run the bot with new RAG info
    remove_previous_RAG_info("abc4")

    requirementsPrompt = getRequirementsPrompt(requirements)
    searchResultsPrompt = getSearchResultsPrompt(searchResults)

    response = with_message_history.invoke(
        { "messages": [
            (   "system",
                requirementsPrompt
            ),
            (   "system",
                searchResultsPrompt
            ),
            HumanMessage(content=input)]
        },
        config=config,
    )
    
    return response.content

In [114]:
import ipywidgets as widgets
from IPython.display import display

# Create input widgets
text_input = widgets.Text()

# Create a button
submit_button = widgets.Button(description='Submit')


# Define a function to handle form submission
def handle_submit(button):
    response = getResponse(text_input.value)
    print(response)

# Attach the function to the button's on_click event
submit_button.on_click(handle_submit)

# Display the form
display(text_input, submit_button)


Text(value='')

Button(description='Submit', style=ButtonStyle())

In [115]:
#| hide
from nbdev import nbdev_export
nbdev_export()